In [ ]:
import xlrd
import re
from random import randrange

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Read file
def read_file(file_loc):
  wb = xlrd.open_workbook(file_loc) 
  sheet = wb.sheet_by_index(0)
  raw_data = []
  for i in range(sheet.nrows): 
      raw_data.append(sheet.cell_value(i, 0))
  return raw_data

In [ ]:
#Pre-processing i.e removing noise from data
def pre_processing(raw_data):
  word_removal_list = [("؛ ٫ / - . \ ! ؟ ۔ ٪ a b c d e f g h i j k l m n o p q r s t u v w x y z 0 1 2 3 4 5 6 7 8 9".split())]
  refined_data = []
  for i in range(len(raw_data)):
    punctuation_removed = []
    for j in range(len(word_removal_list[0])):
      if (j==0):
        punctuation_removed.append(raw_data[i].replace(word_removal_list[0][j],''))
        punctuation_removed.append(punctuation_removed[-1].replace('،',' ، '))
      else:
        punctuation_removed.append(punctuation_removed[-1].replace(word_removal_list[0][j],''))
    refined_data.append(punctuation_removed[-1])
  return refined_data

In [ ]:
#Tokenization
def generate_token(refined_data):
  tokenz = []
  for i in range(len(refined_data)):
    tokenz.append(re.findall(r'[،^آ-ۓ][^ ]*',refined_data[i])) # Find all the words which are separated by space and tokenize
  return tokenz

# For Standard n-grams
def standard_tokenize(refined_data):
  ngram_token = []
  for i in range(len(refined_data)):
    new_data = refined_data[i].replace(' ، ',' ')
    ngram_token.append(re.findall(r'[آ-ۓ][^ ]*',new_data))
  return ngram_token

# For Backward Bigram
def backward_tokenize(normal_token):
  reversed_token = []
  for i in range(len(normal_token)):
    reversed_token.append(normal_token[i][::-1])
  return reversed_token

In [ ]:
# A list of starting words 
# I am only taking the words of first 2 verses of each stanza in the dataset
def generate_starting_words(tokens):
  starting_words = []
  for i in range(len(tokens)):
    if (('،') in tokens[i]):
      starting_words.append(tokens[i][0])
      starting_words.append(tokens[i][tokens[i].index('،')+1])
  return starting_words

def generate_starting_two_words(tokens):
  starting_two_words = []
  for i in range(len(tokens)):
    if (('،') in tokens[i]):
      starting_two_words.append(tokens[i][0]+' '+tokens[i][1])
      starting_two_words.append(tokens[i][tokens[i].index('،')+1]+' '+tokens[i][tokens[i].index('،')+2])
  return starting_two_words

In [ ]:
#Get Unigrams and their count
def generate_unigram(ngram_token):
  unigram = []
  c_unigram = []
  for i in range(len(ngram_token)):
    for j in range(len(ngram_token[i])):
      if (ngram_token[i][j] not in unigram):
        unigram.append(ngram_token[i][j])
        c_unigram.append(1)
      else:
        c_unigram[unigram.index(ngram_token[i][j])] += 1 
  return unigram,c_unigram

In [ ]:
#Unigram probabilities
def generate_unigram_prob(unigram,c_unigram):
  Prob_unigram = []
  for i in range(len(unigram)):
    Prob_unigram.append(c_unigram[i]/len(unigram))
  return Prob_unigram

In [ ]:
#Get Bigrams and their count
def generate_bigram(ngram_token):
  bigram = []
  c_bigram = []
  for i in range(len(ngram_token)):
    for j in range(len(ngram_token[i])-1):
      if ((ngram_token[i][j]+' '+ngram_token[i][j+1]) not in bigram):
        bigram.append(ngram_token[i][j]+' '+ngram_token[i][j+1])
        c_bigram.append(1)
      else:
        c_bigram[bigram.index(ngram_token[i][j]+' '+ngram_token[i][j+1])] +=1
  return bigram,c_bigram

In [ ]:
#bigram probabilities
def generate_bigram_prob(bigram,c_bigram):
  Prob_bigram = []
  for i in range(len(bigram)):
    Prob_bigram.append(c_bigram[i]/len(bigram))
  return Prob_bigram

In [ ]:
#Trigrams and their count
def generate_trigram(ngram_token):
  trigram = []
  c_trigram = []
  for i in range(len(ngram_token)):
    for j in range(len(ngram_token[i])-2):
      if ((ngram_token[i][j]+' '+ngram_token[i][j+1]+' '+ngram_token[i][j+2]) not in trigram):
        trigram.append(ngram_token[i][j]+' '+ngram_token[i][j+1]+' '+ngram_token[i][j+2])
        c_trigram.append(1)
      else:
        c_trigram[trigram.index(ngram_token[i][j]+' '+ngram_token[i][j+1]+' '+ngram_token[i][j+2])] +=1
  return trigram,c_trigram

In [ ]:
#trigram probabilities
def generate_trigram_prob(trigram,c_trigram):
  Prob_trigram = []
  for i in range(len(trigram)):
    Prob_trigram.append(c_trigram[i]/len(trigram))
  return Prob_trigram

In [ ]:
# Generation of verses using bigrams
def generate_stanza_bigram(lenverse,bigram,Prob_bigram,starting_words):
  stanza = []
  for j in range(4): # Size of Stanza
    verse = []
    rand_index = randrange(len(starting_words))
    verse.append(starting_words[rand_index])
    for i in range(lenverse):
      valid_next_word = []
      prob_next_word = []
      given_that_this_word = verse[len(verse)-1] + ' '
      for k in range(len(bigram)):
        if (given_that_this_word in bigram[k]):
          valid_next_word.append(bigram[k])
          prob_next_word.append(Prob_bigram[k])
      next_word_bigram = valid_next_word[prob_next_word.index(max(prob_next_word))]
      extract_word = r' [آ-ۓ]*'
      next_word = re.findall(extract_word,next_word_bigram)
      final = next_word[0].replace(" ","")
      verse.append(final)
    comp_verse = ""
    for x in range(len(verse)):
      if (x!=len(verse)-1):
        comp_verse+=verse[x]+ ' '
      else:
        comp_verse+=verse[x]
    # print(comp_verse)
    stanza.append(comp_verse)
  comp_stanza = ""
  for y in range(len(stanza)):
      comp_stanza+=stanza[y]+'\n'
  return comp_stanza

In [ ]:
# Generation of verses using trigrams
def generate_stanza_trigram(lenverse,trigram,Prob_trigram,Prob_bigram,bigram,starting_words):
  stanza = []
  for j in range(4):
    verse = []
    rand_index = randrange(len(starting_words))
    verse.append(starting_words[rand_index])
    # Get second word using bigram
    valid_next_word = []
    prob_next_word = []
    given_that_this_word = verse[len(verse)-1] + ' '
    for k in range(len(bigram)):
      if (given_that_this_word in bigram[k]):
        valid_next_word.append(bigram[k])
        prob_next_word.append(Prob_bigram[k])
    next_word_bigram = valid_next_word[prob_next_word.index(max(prob_next_word))]
    extract_word_regex = r' [آ-ۓ]*'
    next_word = re.findall(extract_word_regex,next_word_bigram)
    final = next_word[0].replace(" ","")
    verse.append(final)
    # Get other words using trigrams
    for i in range(lenverse):
      valid_next_word_trigram = []
      prob_next_word_trigram = []
      given_that_these_words = verse[len(verse)-2] + ' ' + verse[len(verse)-1] + ' '
      for j in range(len(trigram)):
        if (given_that_these_words in trigram[j]):
          valid_next_word_trigram.append(trigram[j])
          prob_next_word_trigram.append(prob_trigram[j])
      next_word_trigram = valid_next_word_trigram[prob_next_word_trigram.index(max(prob_next_word_trigram))]
      extract_word_regex = r' [آ-ۓ]*'
      next_word = re.findall(extract_word_regex,next_word_trigram)
      final_word = next_word[1].replace(' ','')
      verse.append(final_word)
    comp_verse = ""
    for x in range(len(verse)):
      if (x!=len(verse)-1):
        comp_verse+=verse[x]+ ' '
      else:
        comp_verse+=verse[x]
    # print(comp_verse)
    stanza.append(comp_verse)
  comp_stanza = ""
  for y in range(len(stanza)):
    comp_stanza+=stanza[y]+'\n'
  return comp_stanza

In [ ]:
#Main 
file_loc = ("/content/drive/My Drive/Poetry_Dataset.xlsx")
raw_data = read_file(file_loc)
refined_data = pre_processing(raw_data)

In [ ]:
tokens = generate_token(refined_data)
ngram_token = standard_tokenize(refined_data)
reverse_token = backward_tokenize(ngram_token)
starting_words = generate_starting_words(tokens)

In [ ]:
unigram,count_unigram = generate_unigram(ngram_token)
prob_unigram = generate_unigram_prob(unigram,count_unigram)

In [ ]:
bigram,count_bigram = generate_bigram(ngram_token)
prob_bigram = generate_bigram_prob(bigram,count_bigram)

In [ ]:
reverse_bigram,count_reverse_bigram = generate_bigram(reverse_token)
prob_reverse_bigram = generate_bigram_prob(reverse_bigram,count_reverse_bigram)

In [ ]:
trigram,count_trigram = generate_trigram(ngram_token)
prob_trigram = generate_trigram_prob(trigram,count_trigram)

In [ ]:
lenverse = randrange(3,7) 
stanzas = []
print('Length of each verse is: ',lenverse+1)
for i in range(3):
  stanzas.append(generate_stanza_bigram(lenverse,bigram,prob_bigram,starting_words))
for x in range(3):
  print(stanzas[x])

Length of each verse is:  4
نیا میں نے میں
میں نے میں نے
نیند میں نے میں
خود کو بھی نہیں

تمہارے ساتھ ہیں ہے
زندہ ہوں میں نے
آندھیوں کے لیے ہے
دوستی کا ہے کہ

عشق میں نے میں
لگتی ہے کہ میں
انساں کا ہے کہ
میں نے میں نے



In [ ]:
lenverse_reverse_bigram = randrange(3,7) 
stanzas_reverse_bigram = []
print('Length of each verse is: ',lenverse_reverse_bigram+1)
for i in range(3):
  stanzas_reverse_bigram.append(generate_stanza_bigram(lenverse_reverse_bigram,reverse_bigram,prob_reverse_bigram,starting_words))
for x in range(3):
  print(stanzas_reverse_bigram[x])

Length of each verse is:  4
گہرے کے اس ہے
فسانہ ہی جا کے
آدمی ہے نہیں بھی
ویرانیاں کارواں اور ہے

مذہبی بہت ہے نہیں
روبرو کے اس ہے
یہ ہے نہیں بھی
سمجھے کیا میں ہوں

جب ہے نہیں بھی
تو ہے نہیں بھی
ناک ہے نہیں بھی
مانا جاناں ہے نہیں



In [ ]:
lenverse_trigram = randrange(3,7) 
stanzas_trigram = []
print('Length of each verse is: ',lenverse_trigram+2)
for i in range(3):
  stanzas_trigram.append(generate_stanza_trigram(lenverse_trigram,trigram,prob_trigram,prob_bigram,bigram,starting_words))
for x in range(3):
  print(stanzas_trigram[x])

Length of each verse is:  6
سلگ رہی ہوں میں خود سے
مزہ آتا ھے مجھ سے کم
جنوں کی راہ میں دل کو
اُس کے ہاتھ میں ہے وہ

تاروں کی طرح تھا وہ بھی
کوئی نہیں ہے یہ کس کی
مقتل میں ہیں جینے کی دعا
من میں جھانک کے دیکھوں تری

یہ جو کام رہ گئے ہیں
اکھٹے جی رہے ہیں مگر وہ
خود کو بھی اب سو گیا
کہ میں رو پڑوں مرے ضبط

